# Biprop Tutorial

## 1 - Introduction

Welcome to the `biprop` tutorial. In this module, we will demonstrate the basic usage of the `biprop` library on the example from [this Wikipedia article](https://en.wikipedia.org/wiki/Biproportional_apportionment#Specific_example). If you have not yet heard about biproportional apportionment, it might be a good idea to have a quick look at that Wikipedia page first.

This tutorial assumes that have the python libraries `biprop`, `numpy`, `pandas`, and `matplotlib` installed. If you are missing any of these, you can install all four by running

    > pip install biprop
    > pip install matplotlib

Now we should be good to go. We start with importing the `biprop` library.

In [1]:
import biprop as bp

Now we define the votes, party names and region names from the mentioned Wikipedia example.

In [2]:
votes = [[123,  45, 815],
         [912, 714, 414],
         [312, 255, 215],]
party_names  = ['A', 'B' , 'C'  ]
region_names = ['I', 'II', 'III']

With this we can create an `Election` object. `Election` objects keep track of the votes, the parties and regions involved in the election, and other potentially relevant information.

In [3]:
e = bp.Election(votes, party_names=party_names, region_names=region_names)

The newly created `Election` method has many methods that apply different apportionment methods and return a seat distibution. The first thing that the Wikipedia example does is calculating the upper apportionment. We can use the `upper_apportionment` method to do the same. In this method, we have to specify the total number of seats (20 in our case) and whether we want to calculate the upper apportionment for the parties or the regions (in our case we want both, so we will have to call the method twice).

In [4]:
party_seats  = e.upper_apportionment(total_seats=20, which='parties')
region_seats = e.upper_apportionment(total_seats=20, which='regions')

print('Upper apportionment results:')
print('party seats :', party_seats)
print('region seats:', region_seats)

Upper apportionment results:
party seats : [ 5 11  4]
region seats: [7 5 8]


As a quick sanity check, we can compare these numbers with the ones from Wikipedia and see that we arrived at the same upper apportionment. As previously mentioned, the `Election` object keeps track of additional relevant information. It remembered the total seats and the calculated upper apportionments. We can access this information through the `total_seats`, `party_seats`, and `region_seats` attributes.

In [5]:
print('e.total_seats  =', e.total_seats)
print('e.party_seats  =', e.party_seats)
print('e.region_seats =', e.region_seats)

e.total_seats  = 20
e.party_seats  = [ 5 11  4]
e.region_seats = [7 5 8]


Now we can start with the lower apportionment. Luckily for us, there is a convenient `lower_apportionment` method. We do not even need to pass any arguments since the `Election` object already knows the total seat number and the upper apportionments.

In [6]:
seats = e.lower_apportionment()

Lower apportionment converged after 2 iterations.


The text above informs us that the lower apportionment successfully terminated after two iterations. In an election with $n$ parties and $m$ regions, the method may need up to $n \times m$ iterations to converge. In rare occasions, it may never converge at all. But for now, you do not have to remember this. We will examine these rare cases in a later module. Instead, let us take a look at the calculated seat distribution and compare it to the one from the Wikipedia example.

In [7]:
print('Final seat distribution:')
print(seats)

Final seat distribution:
[[1 0 4]
 [4 4 3]
 [2 1 1]]


Congratulations, you just finished your first biproportional apportionment calculation! And as a bonus, we did indeed receive the same numbers as the Wikipedia article. So we can be reasonably certain that our result is correct.

One more thing: We had to call several methods to perform the biproportional apportionment. But the `Election` object also has a `biproportional_apportionment` method that calculates the upper and lower apportionment in one go. Not only is this more compact, we will also see in a later module that this has additional advantages. Let us create a second `Election` object to demonstrate this more compact way. This time, we specify the total number of seats at the initialization.

In [8]:
e2 = bp.Election(votes, party_names=party_names, region_names=region_names, total_seats=20)
seats2 = e2.biproportional_apportionment()

ValueError: Cannot use `party_seats=None` while `self.party_seats==None`.

This did not work. Apparently we cannot use the keyword argument `party_seats=None` while `e2.party_seats` is `None`. Let us examine why this is.

There are different ways to calculate the upper apportionment. Often, the upper apportionment is not even calculated based on the votes, but rather is some predefined distribution that was calculated based on something else (e.g. the population of the regions). Whenever we call the `biproportional_apportionment` or the `lower_apportionment` methods, we need to specify how we want the upper apportionment to be calculated or whether we want to use a predefined distribution. The last time, we specified this implicitly through calling the `upper_apportionment` method. This time, we need to mention it explicilty through the `party_seats` and `region_seats` arguments. The Wikipedia example uses the Sainte-Laguë method for the upper apportionment. This method corresponds to arithmetic rounding (for an overview, which method corresponds to which rounding method, see [this Wikipedia page](https://en.wikipedia.org/wiki/Highest_averages_method#Specific_methods)). We therefore specify that we want to use the Sainte-Laguë method by passing `np.round` to the method (we have to use `np.round` instead of the standard `round` since the rounding function needs to be able to handle `numpy`-arrays). We can therefore use the following code:

In [9]:
import numpy as np
seats2 = e2.biproportional_apportionment(party_seats=np.round, region_seats=np.round)
print('\nFinal seat distribution with the second method:')
print(seats2)
print('\nseats and seats2 are equal:', np.all(seats==seats2))

Lower apportionment converged after 2 iterations.

Final seat distribution with the second method:
[[1 0 4]
 [4 4 3]
 [2 1 1]]

seats and seats2 are equal: True


Now it worked. We also verified that both ways really produce the same result. I hope you learned something and to see you again in module 2.